In [1]:
import os

In [2]:
! pwd

/media/sergei/bfdb2265-9e92-4f39-83f1-69b664b51628/sergey/Kaggle/google_landmark_cl


In [5]:
PATH_INPUT= '/media/sergei/bfdb2265-9e92-4f39-83f1-69b664b51628/sergey/Kaggle/google_landmark/input/'

In [7]:
listed_files = os.listdir(PATH_INPUT+'test/')
listed_files[:3]

['266a134f7207ff8b.jpg', 'd31662f3cf601151.jpg', 'cf51a5b4a137b7b1.jpg']

In [10]:
! mkdir ./input/test_resized_224

In [11]:
from PIL import Image, ImageOps
def resize_image(filename, new_width=224, new_height=224):
    if os.path.exists('./input/test_resized_224/'+filename):
        return 0
    try:
        pil_image = Image.open(PATH_INPUT+'test/'+filename)
        pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
        pil_image_rgb = pil_image.convert('RGB')
        pil_image_rgb.save('./input/test_resized_224/'+filename, format='JPEG', quality=90)
        return 1
    except:
        return 0

In [12]:
import multiprocessing
import tqdm
pool = multiprocessing.Pool(processes=7)  # Num of CPUs
failures = sum(tqdm.tqdm(pool.imap_unordered(resize_image, listed_files), total=len(listed_files)))
print('Total number of resized impage:', failures)
pool.close()
pool.terminate()

100%|██████████| 115370/115370 [1:21:59<00:00, 23.45it/s]

Total number of resized impage: 115370


In [11]:
res_images = os.listdir('input/train_resized_224/')
len(res_images)

1217308

In [9]:
import pandas as pd

In [10]:
data = pd.read_csv('./input/train.csv')
print(data.shape)
data.head()

(1225029, 3)


,id,url,landmark_id
0,cacf8152e2d2ae60,http://static.panoramio.com/photos/original/70...,4676
1,0a58358a2afd3e4e,http://lh6.ggpht.com/-igpT6wu0mIA/ROV8HnUuABI/...,6651
2,6b2bb500b6a38aa0,http://lh6.ggpht.com/-vKr5G5MEusk/SR6r6SJi6mI/...,11284
3,b399f09dee9c3c67,https://lh3.googleusercontent.com/-LOW2cjAqubA...,8429
4,19ace29d77a5be66,https://lh5.googleusercontent.com/-tnmSXwQcWL8...,6231


In [8]:
data.landmark_id.nunique()

14951

In [11]:
! mkdir ./input/train_resized_224/train
! mkdir ./input/train_resized_224/val

In [13]:
for i in data.landmark_id.unique():
    ! mkdir ./input/train_resized_224/train/$i
    ! mkdir ./input/train_resized_224/val/$i

In [11]:
import numpy as np

In [12]:
def moving_file(file):
    d = np.random.rand()
    folder = data.loc[data.id==file[:-4], 'landmark_id'].values[0]
    if d>.8:
        ! mv ./input/train_resized_224/$file ./input/train_resized_224/val/$folder/$file 
    else:
        ! mv ./input/train_resized_224/$file ./input/train_resized_224/train/$folder/$file 
    return 1

In [13]:
import multiprocessing
import tqdm
pool = multiprocessing.Pool(processes=8)  # Num of CPUs
failures = sum(tqdm.tqdm(pool.imap_unordered(moving_file, listed_files), total=len(listed_files)))
print('Total number of resized impage:', failures)
pool.close()
pool.terminate()

100%|██████████| 535372/535372 [4:59:28<00:00, 29.80it/s]  

Total number of resized impage: 535372
